# __AOE2RECS API modules__

## Load libraries and set options/environment

In [1]:
import requests
import json
import pandas as pd

In [12]:
with open("/Users/Parikshit_verma/Documents/Github/AgeofEmpires2_Voobly/Input/aoe2recs_match_data_query.txt", "r", encoding="utf-8") as text:
    textfile = text.read()
    print(textfile)

query {
    match(id: 51560) {
    id
    map_name
    played
    added
    has_playback
    victory_condition
    duration
    speed
    map_size
    rated
    diplomacy_type
    team_size
    cheats
    population_limit
    lock_teams
    mirror
    version
    game_version
    save_version
    build
    postgame
    platform_match_id
    winning_team{
      team_id
      winner
      players{
        name
      }
    }
    losing_team{
      team_id
      winner
      players{
        name
      }
    }
    files{
      original_filename
      size
      language
      encoding
      owner{
        name
      }
      download_link
    }
    difficulty
    type
    map_reveal_choice
    dataset{
      name
    }
    platform{
      name
      url
      match_url
    }
    ladder{
      name
    }
    event{
      year
      name
    }
    tournament{
      name
    }
    series{
      name
    }
    minimap_link
    odds{
      teams{
        wins
        losses
      }
    civilizat

In [10]:
query

<_io.TextIOWrapper name='/Users/Parikshit_verma/Documents/Github/AgeofEmpires2_Voobly/Input/aoe2recs_match_data_query.txt' mode='r' encoding='utf-8'>

In [4]:
url = 'https://aoe2recs.com/api'
r = requests.post(url, json={'query': query})
print(r.status_code)
print(r.text)

200
{"data":{"match":{"id":51560,"map_name":"Arabia","played":"2019-04-13 17:33:46.597872","added":"2019-04-13 18:25:48.159331","has_playback":null,"victory_condition":"Standard","duration":"0:25:27","speed":"Standard","map_size":"Tiny","rated":true,"diplomacy_type":"1v1","team_size":"1v1","cheats":false,"population_limit":200,"lock_teams":true,"mirror":false,"version":"Userpatch 1.5","game_version":"VER 9.F","save_version":"11.76","build":null,"postgame":true,"platform_match_id":"19640830","winning_team":{"team_id":0,"winner":true,"players":[{"name":"__Ema__"}]},"losing_team":null,"files":[{"original_filename":"rec.20190413-141901.mgz","size":1352945,"language":"es","encoding":"latin-1","owner":{"name":"__Ema__"},"download_link":"/api/download/51984"}],"difficulty":"Easiest","type":"Random Map","map_reveal_choice":"Normal","dataset":{"name":"The Conquerors"},"platform":{"name":"Voobly","url":"https://www.voobly.com","match_url":"https://www.voobly.com/match/view/"},"ladder":{"name":"R